In [2]:
# from data_multi import Bandit_multi
# from learner_diag import NeuralUCBDiag
import numpy as np
import argparse
import pickle
import os
import time
import torch
import numpy as np
from scipy.stats import truncnorm
import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
device
# torch.set_num_threads(8)
# torch.set_num_interop_threads(8)



def truncated_gaussian(mean, variance, a, b, size):
    # Calculate the standard deviation from the variance
    std_dev = np.sqrt(variance)

    # Calculate the lower and upper bounds for truncation
    lower_bound = (a - mean) / std_dev
    upper_bound = (b - mean) / std_dev

    # Generate samples from the truncated normal distribution
    samples = truncnorm.rvs(lower_bound, upper_bound, loc=mean, scale=std_dev, size=size)

    return np.array(samples)




class Evaluation:

    def __init__(self, game_name, task, n_folds, horizon, game, label, context_type):
        self.game_name = game_name
        self.task = task
        self.n_folds = n_folds
        self.horizon = horizon
        self.game = game
        self.label =  label
        self.context_type = context_type
        

    def get_outcomes(self, game, ):
        outcomes = np.random.choice( game.n_outcomes , p= list( game.outcome_dist.values() ), size= self.horizon) 
        return outcomes

    def get_feedback(self, game, action, outcome):
        return game.FeedbackMatrix[ action ][ outcome ]

    def eval_policy_once(self, alg, game, job):

        alg.reset()

        context_generator, jobid = job

        np.random.seed(jobid)

        cumRegret =  np.zeros(self.horizon, dtype =float)
        compteur = None
        for t in range(self.horizon):
            compteur = t
            if t % 1000 == 0 :
                print(t)

            context,distribution = context_generator.get_context()
            outcome = np.random.choice( 2 , p = distribution )

            context = context 
            action = alg.get_action(t, context)

            print('t', t, 'action', action, 'outcome', outcome, 'gaps', game.LossMatrix[0,...] @ distribution - game.LossMatrix[1,...] @ distribution  )
            print()
            feedback =  self.get_feedback( game, action, outcome )

            alg.update(action, feedback, outcome, t, context )

            i_star = np.argmin(  [ game.LossMatrix[i,...] @ np.array( distribution ) for i in range(alg.N) ]  )
            loss_diff = game.LossMatrix[action,...] - game.LossMatrix[i_star,...]
            val = loss_diff @ np.array( distribution )
            cumRegret[t] =  val

        return cumRegret
    
class ToyContexts:

    def __init__(self, ):
        self.type = 'toy'
        self.d_context = 2

    def get_context(self, ):
        p = 1 if np.random.uniform(0, 1)>0.5 else 0 
        return np.array([1,p]).reshape(2,1), [p, 1-p]


In [ ]:
d = 10
sigma_i = 2
lbd = 0.05
value1, value2, value3, value4 = [], [], [], []
for t in range(1, 10000):
    value1.append(  d * (  np.sqrt(  d * np.log(t) + 2 * np.log(1/t**2)   ) + 2 ) )
    value2.append(  d * (  np.sqrt(  (d+1) * np.log(t)   ) + 2 ) )
    value3.append(  sigma_i * (  np.sqrt(  d * np.log(t) + 2 * np.log(1/t**2)   ) +  sigma_i ) )
    value4.append(  sigma_i * (  np.sqrt(  d * np.log(t) + 2 * np.log(1/t**2)   ) + np.sqrt(lbd) * sigma_i ) )

plt.plot( range(9999), value1, label = 'width1')
plt.plot( range(9999), value2, label = 'width2')
plt.plot( range(9999), value3, label = 'width3')
plt.plot( range(9999), value4, label = 'width4')
plt.legend()

In [ ]:
p = 435
sigma_i = 2
lbd = 0.05
value3, value4 = [], []
for t in range(1, 10000):
    value3.append(  sigma_i * (  np.sqrt(  p * np.log(t) + 2 * np.log(1/t**2)   ) +  sigma_i ) )
    value4.append(  sigma_i * (  np.sqrt(  p * np.log(t) + 2 * np.log(1/t**2)   ) + np.sqrt(lbd) * sigma_i ) )

plt.plot( range(9999), value3, label = 'width3')
plt.plot( range(9999), value4, label = 'width4')
plt.legend()

In [ ]:
eta = 1.01


In [ ]:
size = 10
w = np.array([1/size]*size)
context_generator = synthetic_data.LinearContexts( w,'balanced' )

dis = []
for _ in range(1000):
    c,d = context_generator.get_context()
    dis.append(d[0])

plt.hist(dis, bins=30, density=True, alpha=0.7)

In [3]:
import neuralcbpside_v1
import neuralcbpside_v2
import games
import synthetic_data
import numpy as np
import geometry_v3
import cbpside

horizon = 1000
lbd = 1
game = games.apple_tasting( )

size = 5
w = np.array([1/size]*size)
context_generator = synthetic_data.LinearContexts( w,'imbalanced' )

# w = np.array([0,1])
# context_generator = ToyContexts()


# alg = cbpside.CBPside( game, len(w), 1.01, lbd )
# eval1 = Evaluation(None, None, None, horizon, game, None, None)
# job1 = context_generator, 0
# cumRegret1 = eval1.eval_policy_once(alg, game, job1)

alg = neuralcbpside_v1.NeuralCBPside(game, len(w), 1.01, lbd, 20 )
eval2 = Evaluation(None, None, None, horizon, game, None, None)
job2 = context_generator, 0
cumRegret3 = eval2.eval_policy_once(alg, game, job2)

0
t 0 action 0 outcome 0 gaps 0.6527247375416398

t 1 action 1 outcome 1 gaps 0.6992744976791685

det_Zt 1
inside_sqrt 2.772588722239781
1.0 2.6651092223153956 0.0
factor 2.6651092223153956 width [[0.27721825]]
det_Zt 1
inside_sqrt 2.772588722239781
1.0 2.6651092223153956 0.0
factor 2.6651092223153956 width [[0.12944628]]
confidence [array([[0.7388169]], dtype=float32), array([[0.34498847]], dtype=float32)]
estimates [array([[0.57417524]], dtype=float32), array([[ 0.4491173 ],
       [-0.02056465]], dtype=float32)]
pair [0, 1] tdelta -0.46968195401132107 confidence 0.34498847
union1 [0]
t 2 action 0 outcome 1 gaps -0.46390234929033314

136 9.644050643239981e-07 1e-06 False


/home/mheuillet/Desktop/neuralCBPside/.venv/lib/python3.8/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([2, 2])) that is different to the input size (torch.Size([1, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


det_Zt 1.0768499071923114
inside_sqrt 4.4684891811964835
1.0 3.1138801246041563 0.0
factor 3.1138801246041563 width [[0.19116753]]
det_Zt 1
inside_sqrt 4.394449154672439
1.0 3.09629414793641 0.0
factor 3.09629414793641 width [[0.12698229]]
confidence [array([[0.5952728]], dtype=float32), array([[0.3931745]], dtype=float32)]
estimates [array([[0.69126695]], dtype=float32), array([[ 0.51080954],
       [-0.00407794]], dtype=float32)]
pair [0, 1] tdelta -0.5148874786682427 confidence 0.3931745
union1 [0]
t 3 action 0 outcome 0 gaps 0.6740369236492549

129 9.61038597388202e-07 1e-06 False
det_Zt 1.143407746239115
inside_sqrt 5.679190499045571
1.0 3.383105221983614 0.0
factor 3.383105221983614 width [[0.14014111]]
det_Zt 1
inside_sqrt 5.545177444479562
1.0 3.3548200450309493 0.0
factor 3.3548200450309493 width [[0.12536804]]
confidence [array([[0.47411215]], dtype=float32), array([[0.4205872]], dtype=float32)]
estimates [array([[0.65435094]], dtype=float32), array([[ 0.41212386],
       [-0

KeyboardInterrupt: 

In [ ]:
alg.functionnal[1]['p']

In [ ]:
# plt.plot(range(horizon), np.cumsum(cumRegret1) )
# plt.plot(range(horizon), np.cumsum(cumRegret2) )
plt.plot(range(horizon), np.cumsum(cumRegret3) )

In [1]:
5 * 1e-6

4.9999999999999996e-06

In [ ]:
c,d = context_generator.get_context()
c - np.ones((5,1)) * 0.5

In [ ]:
# alg.contexts[1]['labels']

action = 1

Y_it = np.array( alg.contexts[action]['labels'] )
X_it =  np.array( alg.contexts[action]['features'] )

Y_it =  np.squeeze(Y_it, 2).T # Y_it.reshape( (sigma, n) )
X_it =  np.squeeze(X_it, 2).T #X_it.reshape( (d, n) )

means = np.mean( X_it, 1 )

X_prime_it = X_it - means[:, np.newaxis]

norms = np.linalg.norm(X_prime_it, np.inf, axis=0, keepdims=True)

# Divide each element by its L2 norm
X_prime2_it = X_prime_it / norms

In [ ]:
from sklearn.linear_model import Ridge
import numpy as np

symb = 0


### data not centered and not scaled
clf = Ridge(alpha=0.01, fit_intercept = True)
clf.fit(X_it.T, Y_it[symb])
print(clf.coef_)
print(clf.intercept_) #@ context
print()
clf = Ridge(alpha=0.01, fit_intercept = False)
clf.fit(X_it.T, Y_it[symb])
print(clf.coef_)
print(clf.intercept_) #@ context

print()
print(Y_it[symb] @ X_it.T @ alg.contexts[1]['V_it_inv'])
print()
### data centered and not scaled

clf = Ridge(alpha=0.01, fit_intercept = True)
clf.fit(X_prime_it.T, Y_it[symb])
print(clf.coef_)
print(clf.intercept_) #@ context
print()
clf = Ridge(alpha=0.01, fit_intercept = False)
clf.fit(X_prime_it.T, Y_it[symb])
print(clf.coef_)
print(clf.intercept_) #@ context
print()
### data centered and scaled
clf = Ridge(alpha=0.01, fit_intercept = True)
clf.fit(X_prime2_it.T, Y_it[symb])
print(clf.coef_)
print(clf.intercept_) #@ context
print()
clf = Ridge(alpha=0.01, fit_intercept = False)
clf.fit(X_prime2_it.T, Y_it[symb])
print(clf.coef_)
print(clf.intercept_) #@ context

In [ ]:
from sklearn.linear_model import Ridge
import numpy as np




In [ ]:
import numpy as np
alg.contexts[0]['weights'] #@ context

In [ ]:
game = games.apple_tasting()
# outcome = 0
# action = 1
# M = 2
# e_y = np.zeros( (M, 1) )
# e_y[outcome] = 1
# Y_t =  game.SignalMatrices[action] @ e_y 
# print(Y_t)
# print( game.SignalMatrices[action].shape , e_y.shape)

N = 2
M = 2
mathcal_N = [ [0, 1], [1, 0] ] 

import geometry_v3

halfspace = [ (  [0,1], 1) ]

Pt = geometry_v3.getParetoOptimalActions(game.LossMatrix, N, M, halfspace)

Nt = geometry_v3.getNeighborhoodActions(game.LossMatrix, N, M, halfspace,  mathcal_N )

print(Pt)
print(Nt)


In [ ]:
c = np.random.uniform(0,1,10)
print( np.linalg.norm(c)  )
c_prime = c / np.linalg.norm(c) 
print( np.linalg.norm(c_prime)  )

In [ ]:

plt.plot(range(horizon), np.cumsum(cumRegret1), label = 'NeuralCBPside' )
# plt.plot(range(horizon), np.cumsum(cumRegret2), label = 'CBPside' )
plt.legend()
# plt.ylim(0,10000)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from functools import partial
_ = torch.manual_seed(0)

class Network(nn.Module):
    def __init__(self, output_dim, dim, hidden_size=10):
        super(Network, self).__init__()
        self.fc1 = nn.Linear(dim, hidden_size)
        self.activate = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_dim)
    def forward(self, x):
        return self.fc2(self.activate(self.fc1(x)))

func = Network( 1, 5, 5 )

# Create a tensor representing the input variables
x = torch.tensor([2.0, 3.0, 1.0, 1.0, 0.5], requires_grad=True)

pred = func(x)

func.zero_grad()

pred.backward()

# Access the gradient
gradient = x.grad

print(gradient)

g = torch.cat([p.grad.flatten().detach() for p in func.parameters()])

print(g, len(g))

total_params = sum( p.numel() for p in func.parameters() if p.requires_grad)
total_params

In [ ]:
#### linear task:

class LinearContexts:
    def __init__(self,  w, task):
        self.d = len(w) # number of features
        self.w = w
        self.type = 'linear'
        self.task = task
    
    def get_context(self, ):

        if self.task == 'balanced':
            context = truncated_gaussian(0, 0.1, 0, 1, size) if np.random.uniform(0,1)<0.5 else truncated_gaussian(1, 0.1, 0, 1, size)
        else:
            context = truncated_gaussian(0.5, 1, 0, 1, size)
        cont = np.array(context).reshape(self.d,1)
        val = self.w @ cont
        val = [ val[0], 1-val[0] ]
        return cont, val 

size = 5
w = np.array([1/size]*size)
balanced, imbalanced = LinearContexts(w, 'balanced') ,LinearContexts(w, 'imbalanced')
values1, values2 = [], []    
for _ in range(1000):
    cont1, dist1 = balanced.get_context()
    values1.append( dist1[0] )

    cont2, dist2 = imbalanced.get_context()
    values2.append( dist2[0] )
    # values2.append( w @ balanced.get_context() )

plt.hist(values1, bins=30, density=True, alpha=0.7)
plt.hist(values2, bins=30, density=True, alpha=0.7)
plt.xlabel('Values')
plt.ylabel('Density')
plt.title('Histogram of values1')
plt.show()

In [ ]:
#### quadratic task:

class QuadraticContexts:
    def __init__(self,  w, task):
        self.d = len(w) # number of features
        self.w = w
        self.type = 'quadratic'
        self.task = task
    
    def get_context(self, ):

        if self.task == 'balanced':
            context = truncated_gaussian(0, 0.05, 0, 1/2, size) if np.random.uniform(0,1)<0.5 else truncated_gaussian(1, 0.05, 0, 1/2, size)
        else:
            context = truncated_gaussian(0.5, 1, 0, 1/2, size)
            
        cont = np.array(context).reshape(self.d,1)
        val = 2 * self.w @ cont**2 + self.w @ cont
        val = [ val[0], 1-val[0] ]
        return cont, val 

size = 5
w = np.array([1/size]*size)

balanced, imbalanced = QuadraticContexts(w, 'balanced') ,QuadraticContexts(w, 'imbalanced')
values1, values2 = [], []    
for _ in range(1000):
    cont1, dist1 = balanced.get_context()
    values1.append( dist1[0] )

    cont2, dist2 = imbalanced.get_context()
    values2.append( dist2[0] )
    # values2.append( w @ balanced.get_context() )

plt.hist(values1, bins=30, density=True, alpha=0.7)
plt.hist(values2, bins=30, density=True, alpha=0.7)
plt.xlabel('Values')
plt.ylabel('Density')
plt.title('Histogram of values1')
plt.show()

In [ ]:
#### sinusoidal task:

class CosineContexts:
    def __init__(self,  w, task):
        self.d = len(w) # number of features
        self.w = w
        self.type = 'sinusoidal'
        self.task = task
    
    def get_context(self, ):

        if self.task == 'imbalanced':
            if np.random.uniform(0,1)<0.725:
                context = truncated_gaussian(np.pi/self.d**2, 0.05, 0, np.pi, self.d) if np.random.uniform(0,1)<0.5 else  truncated_gaussian(0, 0.05, 0, np.pi, self.d)
            else :
                context = truncated_gaussian(np.pi/2, self.d * np.pi, 0, np.pi, self.d)
                
        
        else:
            context = truncated_gaussian(np.pi/6, 0.1, 0, np.pi, self.d) if np.random.uniform(0,1)<0.5 else  truncated_gaussian(5 * np.pi / 6, 0.1, 0, np.pi, self.d)
        
        
        cont = np.array(context).reshape(self.d,1)
        val = np.sin(w@cont)
        val = [ val[0], 1-val[0] ]
        return cont, val 

size = 5
w = np.array([1/size]*size)

balanced, imbalanced = CosineContexts(w, 'balanced') ,CosineContexts(w, 'imbalanced')
values1, values2 = [], []    
for _ in range(10000):
    cont1, dist1 = balanced.get_context()
    values1.append( dist1[0] )

    cont2, dist2 = imbalanced.get_context()
    values2.append( dist2[0] )

plt.hist(values1, bins=30, density=True, alpha=0.7, label = 'balanced')
plt.hist(values2, bins=30, density=True, alpha=0.7, label = 'imbalanced')
plt.legend()
plt.xlabel('Values')
plt.ylabel('Density')
plt.title('Histogram of values1')
plt.show()